In [1]:
# Anil Kumar Ravuru

import os
import pandas as pd
import json
from googletrans import Translator
import time
import progressbar

def translate_tweet(tweet_json):
    return True
    global tweets_translated
    if tweet_json['lang'] != 'en':
        try:
            trans = Translator().translate(tweet_json['text'], dest='en')
            tweet_json['text'], tweet_json['lang'] = trans.text, tweet_json['lang'] + ' - en'
            tweets_translated += 1
        except:
            with open('FailedTranslations.txt', 'a') as logFile:
                logFile.write(tweet_json['text'] + '\n--------------------\n')
            return False
    return True

def is_desired_tweet(tweet_json):
    if 'delete' in tweet_json:
        return False
    if 'lang' not in tweet_json:
        return False
    if is_pokemon_in_text(tweet_json['text']):
        return True
    return False

def is_pokemon_in_text(text):
    # pokemon_words = ['pokemongo', 'pokémongo']     # Add whaterev is needed here
    key_words = ['pokemon', 'pokecoins', 'pokmon', 'twitter', 'facebook', 'tinder', 'snapchat', 'amazon', 
                 'clash', 'reigns', 'renegade', 'asphalt', 'pinout', 'sudoku', 'mario', 'dues', 
                 'leagueoflegends', 'dota', 'templerun', 'flappybird']
    temp_text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    temp_text = temp_text.lower().replace(' ', '').replace('é', 'e')
    for word in key_words:
        if word in temp_text:
            return True
    return False

def tweet_parser(tweet_json):
    wanted_fields = ['created_at', 'id', 'text', 'source', 'user', 'geo', 'coordinates', 'place',
                     'retweet_count', 'favorite_count', 'entities', 'lang', 'retweeted_id']
    return {field: tweet_json[field] for field in wanted_fields if field in tweet_json}

# This will return 2 DataFrames. One contining Tweets, another one containing info of retweets.
def read_json_to_dfs(file_name):
    tweet_array, retweet_array = [], []
    with open(file_name, 'r') as tweets:
        for tweet in tweets:
            tweet_json = json.loads(tweet)
            if 'retweeted_status' in tweet_json:
                if not (is_desired_tweet(tweet_json) or is_desired_tweet(tweet_json['retweeted_status'])):
                    continue
                # flatten the tweet_json and add it to retweet_array, retweeted_status to tweet_array
                if translate_tweet(tweet_json):
                    tweet_json['retweeted_id'] = tweet_json['retweeted_status']['id']
                    retweet_array.append(tweet_parser(tweet_json))
                if translate_tweet(tweet_json['retweeted_status']):
                    tweet_array.append(tweet_parser(tweet_json['retweeted_status']))
            else:
                if not is_desired_tweet(tweet_json):
                    continue
                # flatten the tweet_json and add it to tweet_array
                if translate_tweet(tweet_json):
                    tweet_array.append(tweet_parser(tweet_json))
    return pd.DataFrame(tweet_array), pd.DataFrame(retweet_array)

def preprocess_dataframe(df):
    # df = df[df['lang'] == 'en']
    return df

def main():
    global tweets_translated
    all_files = [os.path.join(dp, f) for dp, dn, fn in os.walk('OctData') for f in fn if os.path.splitext(f)[1] == '.bz2']
    pokemon_tweet_count, pokemon_retweet_count = 0, 0
    pokemon_tweet_df, pokemon_retweet_df = pd.DataFrame([]), pd.DataFrame([])
    for file_index in progressbar.progressbar(range(len(all_files))):
    # for file_index in range(len(all_files)):
        file_name = all_files[file_index]
        try:
            os.system('bzip2 -dk '+ file_name)             # Extract bz2 file
            tweet_temp_df, retweet_temp_df = read_json_to_dfs(file_name[:-4])
            preprocess_tweet_df = preprocess_dataframe(tweet_temp_df)
            preprocess_retweet_df = preprocess_dataframe(retweet_temp_df)
            with open('LogFile.txt', 'a') as logFile:
                logFile.write(file_name[:-4]+'\n')
        except Exception as e:
            with open('NegativeLogFile.txt', 'a') as negativeLogFile:
                negativeLogFile.write(file_name + ' ' + str(e) + '\n')
        if not tweet_temp_df.empty:
            pokemon_tweet_count += tweet_temp_df.count().id
            pokemon_tweet_df = pokemon_tweet_df.append(tweet_temp_df, sort=False, ignore_index=True)
            # print('tweet count: ', tweet_temp_df.count().id)
        if not retweet_temp_df.empty:
            pokemon_retweet_count += retweet_temp_df.count().id
            pokemon_retweet_df = pokemon_retweet_df.append(retweet_temp_df, sort=False, ignore_index=True)
            # print('retweet count: ', retweet_temp_df.count().id)
        os.system('rm -f ' + file_name[:-4])          # Delect the extracted json file

    print('Tweets:  ', pokemon_tweet_count)
    print('Retweets: ', pokemon_retweet_count)
    pokemon_tweet_df.to_csv('Tweets.csv', sep=',', index=False)
    pokemon_retweet_df.to_csv('Reweets.csv', sep=',', index=False)
    return

tweets_translated = 0
if __name__ == '__main__':
    main()
    print('Translated tweet count: ', tweets_translated)


  0% (26 of 37533) |                     | Elapsed Time: 0:00:19 ETA:   7:10:23

KeyboardInterrupt: 

In [2]:
    
# df_temp = pd.read_json('01/01.json', lines=True)
# df_temp = df_temp.dropna(subset=['text'])
# print(df_temp.count().text)


# start_time = int(time.time()*1000.0)
# df_temp['language'] = df_temp.loc[:,'text'].apply(lambda x: detect_lang(x))
# df_temp = df_temp[df_temp['language'] == 'en']
# print(df_temp.count().text)
# end_time = int(time.time()*1000.0)
# print('Time taken: ', end_time - start_time)

In [ ]:
# def shit_happens(text):
#     try:
#         return 'pokemon' in str(x).lower()
#     except:
#         return False
# print(df_temp['text'].apply(lambda x: shit_happens(x)))
# df_temp = df_temp[df_temp['text'].apply(lambda x: shit_happens(x))]
# print(df_temp.loc[:, ['text']])